In [33]:
from docplex.cp.model import *
from docplex.cp.config import get_default

In [34]:
from config import setup
setup()

In [35]:
n = 4

F = ['f1', 'f2', 'f3', 'f4']
H = ['h1', 'h2', 'h3', 'h4']

L = {
    "f1":['h2', 'h1', 'h3', 'h4'],
    "f2":['h3', 'h4', 'h1', 'h2'],
    "f3":['h1', 'h3', 'h4', 'h2'],
    "f4":['h2', 'h1', 'h3', 'h4'],

    "h1":['f2', 'f3', 'f1', 'f4'],
    "h2":['f4', 'f1', 'f3', 'f2'],
    "h3":['f2', 'f4', 'f1', 'f3'],
    "h4":['f3', 'f1', 'f4', 'f2']
}

mdl = CpoModel(name="Mariages")

In [36]:
#Lpos[h,f] postition de f dans L(h) + position de h dans L(h)
LPos = {}
for h in H:
    for f in F:
        LPos[h,f]=L[h].index(f)
        LPos[f,h]=L[f].index(h)


In [37]:
#x[h,f] mariage entre h et f
x = {}
for h in H:
    for f in F:
        x[h,f]=mdl.binary_var(name="x_" + h + "_" + f)

#y[i] cout du marriage de i (somme des couts du mariage pour les 2 partis) 
y = {}
for i in H+F:
    y[i]=mdl.integer_var(0,2*(n-1),name="y_"+i)



In [38]:
#Ajout des contraintes

#Contrainte de monogamie
for h in H:
    mdl.add(sum([x[h,f] for f in F])==1)
for f in F:
    mdl.add(sum([x[h,f] for h in H])==1)


#Valeur de y:
for h in H:
    mdl.add(y[h]==sum([(x[h,f]*LPos[h,f]) for f in F]))
for f in F:
    mdl.add(y[f]==sum([(x[h,f]*LPos[f,h]) for h in H]))

#On cherches à minimiser les couts des marriages
mdl.add(minimize(max([y[i] for i in H+F])))

In [39]:
solution = mdl.solve()
solution.print_solution()

 ! ----------------------------------------------------------------------------
 ! Minimization problem - 24 variables, 16 constraints
 ! Workers              = 1
 ! Presolve             = Off
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 38.5 (before), 38.5 (after)
 !  . Memory usage      : 300.8 kB (before), 300.8 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         24                 -
 + New bound is 0
                        0         24                 -
 + New bound is 2
 *             2        1  0.00s               (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : 2 (optimal - effective tol. is 0)
 ! Best bound             : 2
 ! --------------------------

In [40]:
#Affichage des de la solution:
for h in H:
    for f in F:
        if(solution[x[h,f]]==1):
            print("Mariage entre ",h," et ",f," (Couts = ",LPos[h,f],LPos[f,h],")")


Mariage entre  h1  et  f1  (Couts =  2 1 )
Mariage entre  h2  et  f4  (Couts =  0 0 )
Mariage entre  h3  et  f2  (Couts =  0 0 )
Mariage entre  h4  et  f3  (Couts =  0 2 )
